In [1]:
import pandas as pd

In [2]:
diagnosis = pd.read_csv("diagnoses.csv") 

events = pd.read_csv("events.csv")

In [3]:
def diagnosis_for_events(diagnosis, events):
    """Add extra column on the events table representing the most recent diagnosis after that event 
       Add flag columns to mark rows with: 1.Events with no ID, 2.Events with date older than diagnosis
    """
    
    # get diasnosis Time and Date into separate columns, convert strings into datetime format
    time_date = diagnosis["DiagnosisDateTime"].str.split(pat = ' ', expand = True)
    diagnosis[['Diagnosis_time', 'Diagnosis_date']] = time_date
    diagnosis['Diagnosis_date'] =  pd.to_datetime(diagnosis['Diagnosis_date'], format = "%d/%m/%Y")
    diagnosis["Diagnosis_time"] = pd.to_datetime(diagnosis['Diagnosis_time'],format= '%H:%M' ).dt.time
    events['EventDateTime'] =  pd.to_datetime(events['EventDateTime'], format = "%d/%m/%Y")
    
    #keep rows with Nan values for diagnosis, by replacing for 'Not_specified'
    diagnosis['Diagnosis'] = diagnosis['Diagnosis'].fillna('Not_specified')
    
    # get most recent diagnosis for each EventID
    diagnosis = diagnosis.sort_values(["EventID", "Diagnosis_time", "Diagnosis_date"], ascending=[True, False, False])
    latest_diagnosis = diagnosis.drop_duplicates(subset=['EventID'], keep='first')
    del diagnosis
    
    # join diagnosis and event by "EventID"
    output = pd.merge(events, latest_diagnosis[['EventID', 'Diagnosis', "Diagnosis_date"]], how="left", on=['EventID'])
    
    # Flags:
    #"Diagnosis_date_before_event"
    output["Diagnosis_date_before_event"] = output.apply(lambda x: True if x['EventDateTime'] 
                                                         > x['Diagnosis_date'] else False, axis=1)
    
    # "No_EventId"
    output["No_event_id"] = pd.isnull(output['EventID'])
    
    output.drop(columns=['Diagnosis_date'])
    
    return output

In [4]:
output_file = diagnosis_for_events(diagnosis, events).to_csv("events_with_diagnosis.csv", index = False)